In [46]:
from transformers import CvtModel,CvtConfig

configuration = CvtConfig(cls_token=[False,False,False],depth=[1,4,16])

In [47]:
ViT = CvtModel.from_pretrained("microsoft/cvt-21",config=configuration,ignore_mismatched_sizes=True)
print(ViT.config)
len(list(ViT.encoder.stages[2].children())[1])

Some weights of the model checkpoint at microsoft/cvt-21 were not used when initializing CvtModel: ['layernorm.bias', 'classifier.weight', 'classifier.bias', 'layernorm.weight', 'cvt.encoder.stages.2.cls_token']
- This IS expected if you are initializing CvtModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CvtModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CvtConfig {
  "_name_or_path": "microsoft/cvt-21",
  "attention_drop_rate": [
    0.0,
    0.0,
    0.0
  ],
  "cls_token": [
    false,
    false,
    false
  ],
  "depth": [
    1,
    4,
    16
  ],
  "drop_path_rate": [
    0.0,
    0.0,
    0.1
  ],
  "drop_rate": [
    0.0,
    0.0,
    0.0
  ],
  "embed_dim": [
    64,
    192,
    384
  ],
  "initializer_range": 0.02,
  "kernel_qkv": [
    3,
    3,
    3
  ],
  "layer_norm_eps": 1e-12,
  "mlp_ratio": [
    4.0,
    4.0,
    4.0
  ],
  "model_type": "cvt",
  "num_channels": 3,
  "num_heads": [
    1,
    3,
    6
  ],
  "padding_kv": [
    1,
    1,
    1
  ],
  "padding_q": [
    1,
    1,
    1
  ],
  "patch_padding": [
    2,
    1,
    1
  ],
  "patch_sizes": [
    7,
    3,
    3
  ],
  "patch_stride": [
    4,
    2,
    2
  ],
  "qkv_bias": [
    true,
    true,
    true
  ],
  "qkv_projection_method": [
    "dw_bn",
    "dw_bn",
    "dw_bn"
  ],
  "stride_kv": [
    2,
    2,
    2
  ],
  "stride_q": [
    1,
    1,
   

16

In [36]:
from torchvision.io import read_video
vid,_,_ = read_video("/home/c1l1mo/datasets/NACL/1_jumps/1_jump_videos/467204328706015300_0.mp4",pts_unit="sec")
vid.shape

torch.Size([11, 224, 224, 3])

In [5]:
vid = vid.permute(0,3,1,2)

In [6]:
print(vid.shape)
frame = vid[0]
frame.shape

torch.Size([11, 3, 224, 224])


torch.Size([3, 224, 224])

In [7]:
from transformers import AutoImageProcessor
image_processor = AutoImageProcessor.from_pretrained("microsoft/cvt-13")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
2023-11-28 17:19:58.735381: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [8]:
print(frame.shape)
processed_frame = image_processor(vid,return_tensors="pt")

torch.Size([3, 224, 224])


In [9]:
ipt = processed_frame['pixel_values']
ipt.shape

torch.Size([11, 3, 224, 224])

In [10]:
import torch

with torch.no_grad():
    outputs = ViT(ipt)

In [34]:
ViT.encoder

CvtEncoder(
  (stages): ModuleList(
    (0): CvtStage(
      (embedding): CvtEmbeddings(
        (convolution_embeddings): CvtConvEmbeddings(
          (projection): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(2, 2))
          (normalization): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (layers): Sequential(
        (0): CvtLayer(
          (attention): CvtAttention(
            (attention): CvtSelfAttention(
              (convolution_projection_query): CvtSelfAttentionProjection(
                (convolution_projection): CvtSelfAttentionConvProjection(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                )
                (linear_projection): CvtSelfAttentionLinearProjection()
              )


In [13]:
print(outputs.last_hidden_state.shape)

torch.Size([11, 384, 14, 14])


In [14]:
from torch import nn
backbone = nn.Sequential((list(ViT.children())[0]).stages[0],(list(ViT.children())[0]).stages[1],(list((list(ViT.children())[0]).stages[2].children())[0]))
finetune = (list((list(ViT.children())[0]).stages[2].children())[1])

In [17]:
backbone.eval()
finetune.eval()
with torch.no_grad():
    hidden_state,cls_token = (list(ViT.children())[0]).stages[0](ipt)
    hidden_state,cls_token = (list(ViT.children())[0]).stages[1](hidden_state)
    hidden_state = (list((list(ViT.children())[0]).stages[2].children())[0])(hidden_state)
    batch_size, num_channels, height, width = hidden_state.shape
    # rearrange b c h w -> b (h w) c"
    hidden_state = hidden_state.view(batch_size, num_channels, height * width).permute(0, 2, 1)
    for layer in finetune:
        hidden_state = layer(hidden_state,height,width)
    hidden_state = hidden_state.permute(0, 2, 1).view(batch_size, num_channels, height, width)
    print(hidden_state.shape)

torch.Size([11, 384, 14, 14])


In [ ]:
## convert model to onnx
from torch.onnx import TrainingMode
model = CvtModel.from_pretrained("microsoft/cvt-21")
model = model.eval()
dummy_input = torch.rand(40,3,224,224)
input_names = ["input"]
output_names = ["output"]

# torch.onnx.export(model,dummy_input,"CVT_train.onnx",verbose=False,input_names=input_names,output_names=output_names,export_params=True,training=TrainingMode.TRAINING,opset_version=16)